<a href="https://colab.research.google.com/github/lguloy/House-Pricing-Predictions/blob/work_vijitha/Second_try_Feature_importance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
https://finalprojectbootcamp.s3.us-east-2.amazonaws.com/all_dumimies_converted_df.csv

In [1]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
spark_version = 'spark-3.0.3'
# spark_version = 'spark-3.<enter version>'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://security.ubuntu.com/ubuntu bionic-securi

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [3]:
from pyspark import SparkFiles
# Load in train.csv from S3 into a DataFrame
url = "https://finalprojectbootcamp.s3.us-east-2.amazonaws.com/all_dumimies_converted_df.csv"
spark.sparkContext.addFile(url)

df = spark.read.option('header', 'true').csv(SparkFiles.get("all_dumimies_converted_df.csv"), inferSchema=True, sep=',', timestampFormat="mm/dd/yy")
df.show(10)

#Convert to Pandas to allow train_test_split usage
df = df.select("*").toPandas()
df = df.dropna(how="any")

+---+----------+-----------+-------+-----------+-----------+---------+------------+----------+----------+----------+---------+-----------+--------+--------+------------+---------+------------+------------+--------+--------+------------+------------+------------+----------+-----------+----------+----------+----------+-----------+-------------+---------+-----------+--------+-------+------+------+---------+-----------+-----------+-----------+-----------+-----------+----------+------------+------------+------------+---------------+---------------+---------------+----------------+-----------------+-------------+-------------+----------------+-------------+-------------+--------------------+-------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------------+--------------------+--------------------+------------------+--------------------+-------------------+--------------------+-----------------

In [4]:
#Get the Feature names
import pandas as pd

features = [col for col in df.columns]
print(features)

print(type(features))

['Id', 'MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces', 'GarageYrBlt', 'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal', 'MoSold', 'YrSold', 'SalePrice', 'MSZoning_FV', 'MSZoning_RH', 'MSZoning_RL', 'MSZoning_RM', 'Street_Pave', 'Alley_Pave', 'LotShape_IR2', 'LotShape_IR3', 'LotShape_Reg', 'LandContour_HLS', 'LandContour_Low', 'LandContour_Lvl', 'Utilities_NoSeWa', 'LotConfig_CulDSac', 'LotConfig_FR2', 'LotConfig_FR3', 'LotConfig_Inside', 'LandSlope_Mod', 'LandSlope_Sev', 'Neighborhood_Blueste', 'Neighborhood_BrDale', 'Neighborhood_BrkSide', 'Neighborhood_ClearCr', 'Neighborhood_CollgCr', 'Neighborhood_Crawfor', 'Neighborhood_

In [5]:
#Import sklearn tools and dependencies
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [6]:
#####################Create the feature data and the target

#Feature DataFrame is X
X = df.drop(columns=["Id", "SalePrice"])


#Target Dataframe is y
y = df["SalePrice"]


#####################Create train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=2)



In [8]:
###Scale
X_train_scaler = StandardScaler().fit(X_train)
X_test_scaler = StandardScaler().fit(X_test)
X_train_scaled = X_train_scaler.transform(X_train)
X_test_scaled = X_test_scaler.transform(X_test)

In [11]:
rf = RandomForestClassifier(n_estimators=200)
rf.fit(X_train_scaled, y_train)
print(f' Training score: {rf.score(X_train_scaled, y_train)}')
print(f' Test score: {rf.score(X_test_scaled, y_test)}')

 Training score: 1.0
 Test score: 0.028469750889679714


In [13]:
importances = rf.feature_importances_
features.remove("Id")
features.remove("SalePrice")

sorted(zip(rf.feature_importances_, features), reverse=True)

[(0.030092410034162537, 'GrLivArea'),
 (0.02899916606107307, 'LotArea'),
 (0.028606982171723857, '1stFlrSF'),
 (0.02856171795887266, 'TotalBsmtSF'),
 (0.028241855375988417, 'GarageArea'),
 (0.02773424119484836, 'BsmtUnfSF'),
 (0.025962652367270283, 'LotFrontage'),
 (0.025142908493709253, 'MoSold'),
 (0.024918276193296965, 'GarageYrBlt'),
 (0.024059886116785167, 'YearBuilt'),
 (0.023817989683374334, 'YearRemodAdd'),
 (0.023147445056312423, 'BsmtFinSF1'),
 (0.022173565580014796, 'OpenPorchSF'),
 (0.019579426171226575, 'WoodDeckSF'),
 (0.018080451711567185, 'TotRmsAbvGrd'),
 (0.01797239399710295, 'YrSold'),
 (0.01764859331334914, 'MasVnrArea'),
 (0.01674326926077337, '2ndFlrSF'),
 (0.015699381787277207, 'OverallQual'),
 (0.012658359045613026, 'BedroomAbvGr'),
 (0.012329593310759007, 'OverallCond'),
 (0.012192637580604914, 'MSSubClass'),
 (0.009951276652329918, 'Fireplaces'),
 (0.008847719478209011, 'BsmtFullBath'),
 (0.008178000342983374, 'GarageCars'),
 (0.007793235405680407, 'LotConfig_

In [ ]:


features.remove("Id")
features.remove("SalePrice")

sorted(zip(rf.feature_importances_, features), reverse=True)



[(0.03067184507694324, '1stFlrSF'),
 (0.030661811314982232, 'GrLivArea'),
 (0.030157742992857357, 'LotArea'),
 (0.030058219806715736, 'GarageArea'),
 (0.02988531453794777, 'BsmtUnfSF'),
 (0.028733422469003674, 'TotalBsmtSF'),
 (0.026627014082029762, 'LotFrontage'),
 (0.025583719381550338, 'GarageYrBlt'),
 (0.02533056678489419, 'MoSold'),
 (0.025239368282836506, 'YearBuilt'),
 (0.02399383975829967, 'BsmtFinSF1'),
 (0.023615968787044927, 'YearRemodAdd'),
 (0.02222778731027863, 'OpenPorchSF'),
 (0.020104725477762667, 'WoodDeckSF'),
 (0.018763654347624222, 'YrSold'),
 (0.018410230455695117, 'TotRmsAbvGrd'),
 (0.01622040112927497, '2ndFlrSF'),
 (0.015970015071569303, 'MasVnrArea'),
 (0.01564458969957334, 'OverallQual'),
 (0.012276159252435206, 'BedroomAbvGr'),
 (0.01201444710207882, 'MSSubClass'),
 (0.011869620651636485, 'OverallCond'),
 (0.010155158114020211, 'Fireplaces'),
 (0.008291180235079142, 'BsmtFullBath'),
 (0.007851783677985063, 'LotConfig_Inside'),
 (0.007850407737613738, 'Garage